In [1]:
import numpy as np
import json
import pandas as pd
import matplotlib as plt
import seaborn as sns
import csv
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API, Cursor
import requests
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import os

In [2]:
## access token informations 
access_token1 = "1222829174802538496-AXkAccE1dWMjuEFlEYChpgOibc6SbF"
access_token_secret1 = "jtr66CvVvspsANplSBpuMEQR5iLwK2fRtzM6aDhaC1rZT"

consumer_key1 = "sL9E5QL83DVEshfttLjEEj930"
consumer_secret1 = "R9L4Ar7UWTKB4WQw7cEoybumyTInZ6pYP3cQ5GFlWYFllYAeec"

In [3]:
auth = OAuthHandler(consumer_key1, consumer_secret1)
auth.set_access_token(access_token1, access_token_secret1)
api = API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [4]:
class MyStreamListener(StreamListener):
    """
    Twitter listener, collects streaming tweets and output to a file
    """
    def __init__(self, api, OUTPUT_FILENAME, stop_condition=10):
        """
        initialize the stream, with num. of tweets and saving the outputfile
        """
        
        # this line is needed to import the characteristics of the streaming API
        super(MyStreamListener, self).__init__()
        
        # to-count the number of tweets collected        
        self.num_tweets = 0
        
        # save filename
        self.filename = OUTPUT_FILENAME
        
        # stop-condition
        self.stop_condition = stop_condition
        

    def on_status(self, status):
        
        """
        this function runs each time a new bunch of tweets is retrived from the streaming
        """
        
        with open(self.filename, "a+") as f:
            tweet = status._json
            
            f.write(json.dumps(tweet) + '\n')
            #self.output.append(tweet)
            self.num_tweets += 1
        
            # Stop condition        
            if self.num_tweets <= self.stop_condition:
                return True
            else:
                return False
        

    def on_error(self, status):
        """
        function useful to handle errors. It's possible to personalize it 
        depending on the way we want to handle errors
        """
        
        print(status)
        #returning False in on_error disconnects the stream
        return False

In [5]:
OUTPUT_FILENAME = "other-outputs/tweets_US_Election_2020.json"
stop_condition = 500

l = MyStreamListener(api, OUTPUT_FILENAME, stop_condition)
# here we recall the Stream Class from Tweepy to input the authentication info and our personalized listener 
stream = Stream(auth=api.auth, listener=l)


# keywords we may want decide to track 
TRACKING_KEYWORDS = ["Donald", "Trump", "Joe", "Biden", "America", "USA"]
stream.filter(
    track=TRACKING_KEYWORDS, 
    is_async=False, 
    languages = ["en"]
)

In [6]:
json_Election_file = []
for line in open(OUTPUT_FILENAME, 'r'):
    json_Election_file.append(json.loads(line))

In [7]:
dict_train={}
i=0
for train_file in json_Election_file:
    dict_train[i]=train_file
    i+=1
train = pd.DataFrame.from_dict(dict_train, orient='index')
train.reset_index(level=0, inplace=False)

,index,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,lang,timestamp_ms,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,quoted_status_permalink,extended_tweet,display_text_range,extended_entities
0,0,Fri Nov 20 15:32:49 +0000 2020,1329809932888190976,1329809932888190976,RT @EdwardNorton: His ‘turn card’ bluff will b...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,en,1605886369331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Fri Nov 20 15:32:49 +0000 2020,1329809932892262401,1329809932892262401,RT @TeamTrump: Trump Campaign Statement on #Wi...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,en,1605886369332,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Fri Nov 20 15:32:49 +0000 2020,1329809932926017539,1329809932926017539,RT @BrandonStraka: It’s looking more and more ...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,...,en,1605886369340,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Fri Nov 20 15:32:49 +0000 2020,1329809932921622528,1329809932921622528,"RT @mmpadellan: Make no mistake, the cruel act...","<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,...,en,1605886369339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Fri Nov 20 15:32:49 +0000 2020,1329809932921790464,1329809932921790464,RT @joelockhart: Governor Whitmer just told @c...,"<a href=""http://twitter.com/download/android"" ...",False,NaN,None,NaN,...,en,1605886369339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,Fri Nov 20 15:32:49 +0000 2020,1329809933018259456,1329809933018259456,I think this may be right.,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,...,en,1605886369362,NaN,1.329729e+18,1329728889296355328,{'created_at': 'Fri Nov 20 10:10:47 +0000 2020...,"{'url': 'https://t.co/uTs7K7n3Jg', 'expanded':...",NaN,NaN,NaN
6,6,Fri Nov 20 15:32:49 +0000 2020,1329809933026648065,1329809933026648065,"While You Were Focused on The Kraken, the Sena...","<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,en,1605886369364,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,Fri Nov 20 15:32:49 +0000 2020,1329809933093675008,1329809933093675008,RT @realDonaldTrump: We have great support on ...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,en,1605886369380,NaN,1.329753e+18,1329752606298824705,{'created_at': 'Fri Nov 20 11:45:01 +0000 2020...,"{'url': 'https://t.co/ChpkuZvc4s', 'expanded':...",NaN,NaN,NaN
8,8,Fri Nov 20 15:32:49 +0000 2020,1329809933198508036,1329809933198508036,RT @DonnaBlue88: You know why Trump is so pi**...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,NaN,None,NaN,...,en,1605886369405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,Fri Nov 20 15:32:49 +0000 2020,1329809933202649090,1329809933202649090,RT @ProjectLincoln: The people of Michigan hav...,"<a href=""http://twitter.com/download/iphone"" r...",False,NaN,None,NaN,...,en,1605886369406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
pd.set_option('display.max_columns', None)
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 501 entries, 0 to 500
Data columns (total 36 columns):
created_at                   501 non-null object
id                           501 non-null int64
id_str                       501 non-null object
text                         501 non-null object
source                       501 non-null object
truncated                    501 non-null bool
in_reply_to_status_id        70 non-null float64
in_reply_to_status_id_str    70 non-null object
in_reply_to_user_id          76 non-null float64
in_reply_to_user_id_str      76 non-null object
in_reply_to_screen_name      76 non-null object
user                         501 non-null object
geo                          0 non-null object
coordinates                  0 non-null object
place                        5 non-null object
contributors                 0 non-null object
retweeted_status             340 non-null object
is_quote_status              501 non-null bool
quote_count                  

In [9]:
#Normalize functions:
def get_hashtags(data: pd.DataFrame) -> None:
    hashtag_list=[]
    for i in range(len(data)):
        hashtags=[]
        hashtags_info = train['entities'][i]['hashtags']
        for i in range (len(hashtags_info)):
            hashtags.append(hashtags_info[i]["text"])
        hashtag_list.append(hashtags)
    data['hashtags']=hashtag_list

def remove_unnecessary_columns(data: pd.DataFrame)-> pd.DataFrame:
    cols_to_remove=['id', 'id_str', 'display_text_range', 'source', 'truncated', 'in_reply_to_status_id', 
                    'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'geo', 'coordinates', 
                    'place', 'contributors', 'is_quote_status', 'extended_tweet', 'quote_count','lang', 'created_at', 
                    'possibly_sensitive', 'extended_entities', 'quoted_status_id','quoted_status_id_str', 'quoted_status', 
                    'quoted_status_permalink']
    
    data=data.drop(columns=cols_to_remove)
    return data
    
def stemming(text: str) -> None:
    ps = PorterStemmer()
    words = word_tokenize(text) 
    for w in words: 
        print(w, " : ", ps.stem(w)) 
        
def punctuation_removal(text: str) -> None:
    pass

def remove_stop_words(text: str) -> None:
    pass

In [20]:

train['user'][0]

{'id': 63567377,
 'id_str': '63567377',
 'name': 'Johanna Beatty 🇺🇸✌🏼',
 'screen_name': 'yoyobeatty',
 'location': 'PA',
 'url': 'https://joebiden.com/joes-vision/',
 'description': 'Counselor, survivor, victim advocate. Mom to adults. Love & truth are not always nice. #resist 🇺🇸#BidenHarris2020🇺🇸 #BlackLivesMatter',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 1383,
 'friends_count': 4139,
 'listed_count': 0,
 'favourites_count': 42935,
 'statuses_count': 22181,
 'created_at': 'Thu Aug 06 22:59:36 +0000 2009',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': True,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': '8B542B',
 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme8/bg.gif',
 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme8/bg.gif',
 'profile_background_tile': False,
 'profile_link_color': '9D582E',
 'profile_sidebar_bo

In [11]:
#os.remove(OUTPUT_FILENAME) 

In [12]:
#Quitar URLs
#Quitar emojis
#CLEAN .JSON SO IT DOES NOT SUM TWEETS IF RUNED AGAIN